In [1]:
import pandas as pd

df = pd.read_csv("//workspaces/pacc-cricket-analytics/data/raw/25th Oct 2025.csv")

print(df.shape)
df.head(10)

(887, 4)


,Ball,Score,Outcome,Innings
0,60.5,8--160,J Kann to C McFadyen : No Run,1st Innings - Adelaide
1,60.4,8--160,J Kann to C McFadyen : No Run,1st Innings - Adelaide
2,60.3,8--160,J Kann to C McFadyen : No Run,1st Innings - Adelaide
3,60.2,8--160,J Kann to C McFadyen : No Run,1st Innings - Adelaide
4,60.1,8--160,J Kann to C McFadyen : No Run,1st Innings - Adelaide
5,59.7,8--160,T Andrews to A Stockdale : No Run,1st Innings - Adelaide
6,59.6,8--160,T Andrews to A Stockdale : No Run,1st Innings - Adelaide
7,59.5,8--160,T Andrews to A Stockdale : 1nb,1st Innings - Adelaide
8,59.4,8--159,T Andrews to A Stockdale : No Run,1st Innings - Adelaide
9,59.3,8--159,T Andrews to A Stockdale : No Run,1st Innings - Adelaide


In [2]:
print(df.columns)

Index(['Ball', 'Score', 'Outcome', 'Innings'], dtype='str')


In [3]:
# Make a copy so we don't accidentally destroy raw data
df = df.copy()

# Clean column names
df.columns = df.columns.str.lower().str.strip()

df.head()

,ball,score,outcome,innings
0,60.5,8--160,J Kann to C McFadyen : No Run,1st Innings - Adelaide
1,60.4,8--160,J Kann to C McFadyen : No Run,1st Innings - Adelaide
2,60.3,8--160,J Kann to C McFadyen : No Run,1st Innings - Adelaide
3,60.2,8--160,J Kann to C McFadyen : No Run,1st Innings - Adelaide
4,60.1,8--160,J Kann to C McFadyen : No Run,1st Innings - Adelaide


In [4]:
# Extract over and ball number
df["over"] = df["ball"].astype(str).str.split(".").str[0].astype(int)
df["ball_in_over"] = df["ball"].astype(str).str.split(".").str[1].astype(int)

df[["ball", "over", "ball_in_over"]].head(10)

,ball,over,ball_in_over
0,60.5,60,5
1,60.4,60,4
2,60.3,60,3
3,60.2,60,2
4,60.1,60,1
5,59.7,59,7
6,59.6,59,6
7,59.5,59,5
8,59.4,59,4
9,59.3,59,3


In [5]:
# Clean score column
score_split = df["score"].str.replace("--", "-").str.split("-", expand=True)

df["cum_wickets"] = score_split[0].astype(int)
df["cum_runs"] = score_split[1].astype(int)

df[["score", "cum_wickets", "cum_runs"]].head(10)

,score,cum_wickets,cum_runs
0,8--160,8,160
1,8--160,8,160
2,8--160,8,160
3,8--160,8,160
4,8--160,8,160
5,8--160,8,160
6,8--160,8,160
7,8--160,8,160
8,8--159,8,159
9,8--159,8,159


In [6]:
# Sort correctly first (CRITICAL)
df = df.sort_values(by=["innings", "over", "ball_in_over"]).reset_index(drop=True)

# Compute ball-by-ball deltas
df["runs_off_ball"] = df.groupby("innings")["cum_runs"].diff().fillna(0).astype(int)
df["is_wicket"] = df.groupby("innings")["cum_wickets"].diff().fillna(0).astype(int)

# Convert wickets to binary
df["is_wicket"] = df["is_wicket"].clip(lower=0, upper=1)

df[[
    "ball", "score", "runs_off_ball", "is_wicket"
]].head(15)

,ball,score,runs_off_ball,is_wicket
0,0.1,0--0,0,0
1,0.2,0--4,4,0
2,0.3,0--8,4,0
3,0.4,0--8,0,0
4,0.5,0--12,4,0
5,0.6,0--12,0,0
6,1.1,0--12,0,0
7,1.2,0--14,2,0
8,1.3,0--14,0,0
9,1.4,0--14,0,0


In [7]:
innings_check = df.groupby("innings").agg(
    total_runs=("runs_off_ball", "sum"),
    total_wickets=("is_wicket", "sum"),
    balls=("ball", "count")
)

innings_check

,total_runs,total_wickets,balls
innings,,,
1st Innings - Adelaide,160,8,369
1st Innings - Port Adelaide,257,10,518


In [8]:
# Identify common extras
df["is_no_ball"] = df["outcome"].str.contains("nb", case=False, na=False).astype(int)
df["is_wide"] = df["outcome"].str.contains("wide", case=False, na=False).astype(int)

df[["outcome", "runs_off_ball", "is_no_ball", "is_wide"]].head(15)

,outcome,runs_off_ball,is_no_ball,is_wide
0,B Turley to C Carroll : No Run,0,0,0
1,B Turley to C Carroll : 4 runs,4,0,0
2,B Turley to C Carroll : 4 runs,4,0,0
3,B Turley to C Carroll : No Run,0,0,0
4,B Turley to C Carroll : 4 runs,4,0,0
5,B Turley to C Carroll : No Run,0,0,0
6,C Valente to S Scrimgeour : No Run,0,0,0
7,C Valente to S Scrimgeour : 2 runs,2,0,0
8,C Valente to S Scrimgeour : No Run,0,0,0
9,C Valente to S Scrimgeour : No Run,0,0,0


In [9]:
# Create sequential ball index per innings
df["ball_index"] = df.groupby("innings").cumcount() + 1

In [10]:
def assign_innings_phase(over):
    if over <= 10:
        return "Early Overs"
    elif over <= 40:
        return "Middle Overs"
    else:
        return "Late Overs"

df["innings_phase"] = df["over"].apply(assign_innings_phase)

df[["ball", "over", "innings_phase"]].head(30)

,ball,over,innings_phase
0,0.1,0,Early Overs
1,0.2,0,Early Overs
2,0.3,0,Early Overs
3,0.4,0,Early Overs
4,0.5,0,Early Overs
5,0.6,0,Early Overs
6,1.1,1,Early Overs
7,1.2,1,Early Overs
8,1.3,1,Early Overs
9,1.4,1,Early Overs


In [11]:
df.head()

,ball,score,outcome,innings,over,ball_in_over,cum_wickets,cum_runs,runs_off_ball,is_wicket,is_no_ball,is_wide,ball_index,innings_phase
0,0.1,0--0,B Turley to C Carroll : No Run,1st Innings - Adelaide,0,1,0,0,0,0,0,0,1,Early Overs
1,0.2,0--4,B Turley to C Carroll : 4 runs,1st Innings - Adelaide,0,2,0,4,4,0,0,0,2,Early Overs
2,0.3,0--8,B Turley to C Carroll : 4 runs,1st Innings - Adelaide,0,3,0,8,4,0,0,0,3,Early Overs
3,0.4,0--8,B Turley to C Carroll : No Run,1st Innings - Adelaide,0,4,0,8,0,0,0,0,4,Early Overs
4,0.5,0--12,B Turley to C Carroll : 4 runs,1st Innings - Adelaide,0,5,0,12,4,0,0,0,5,Early Overs
